# How to stream events
This guide covers how to stream events from your graph (`stream_mode="events"`). Depending on the use case and user experience of your LangGraph application, your application may process event types differently.

In [13]:
from langgraph_sdk import get_client

client = get_client()

In [27]:
# create thread
thread = await client.threads.create()
thread

{'thread_id': '3f4c64e0-f792-4a5e-aa07-a4404e06e0bd',
 'created_at': '2024-06-24T22:16:29.301522+00:00',
 'updated_at': '2024-06-24T22:16:29.301522+00:00',
 'metadata': {}}

Streaming events produces responses containing an `event` key (in addition to other keys such as `data`). See the LangChain [`Runnable.astream_events()` reference](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.astream_events) for all event types.

In [15]:
# create input
input = {
    "messages": [
        {
            "role": "human",
            "content": "What's the weather in SF?",
        }
    ]
}

# stream events
async for chunk in client.runs.stream(
    thread_id=thread["thread_id"],
    assistant_id="agent",
    input=input,
    stream_mode="events",
):
    print(f"Receiving new event of type: {chunk.event}...")
    print(chunk.data)
    print("\n\n")

Receiving new event of type: metadata...
{'run_id': '1ef301a5-b867-67de-9e9e-a32e53c5b1f8'}



Receiving new event of type: events...
{'event': 'on_chain_start', 'data': {'input': {'messages': [{'role': 'human', 'content': "What's the weather in SF?"}]}}, 'name': 'LangGraph', 'tags': [], 'run_id': '1ef301a5-b867-67de-9e9e-a32e53c5b1f8', 'metadata': {'graph_id': 'agent', 'created_by': 'system', 'run_id': '1ef301a5-b867-67de-9e9e-a32e53c5b1f8', 'user_id': '', 'thread_id': '7196a3aa-763c-4a8d-bfda-12fbfe1cd727', 'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca'}, 'parent_ids': []}



Receiving new event of type: events...
{'event': 'on_chain_start', 'data': {}, 'name': 'agent', 'tags': ['graph:step:6'], 'run_id': '7bb08493-d507-4e28-b9e6-4a5eda9d04f0', 'metadata': {'graph_id': 'agent', 'created_by': 'system', 'run_id': '1ef301a5-b867-67de-9e9e-a32e53c5b1f8', 'user_id': '', 'thread_id': '7196a3aa-763c-4a8d-bfda-12fbfe1cd727', 'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca', 'la

## Token-by-Token Streaming

Token-by-token streaming can be implemented with the `events` streaming mode. The `on_chat_model_stream` event type should be processed to stream LLM responses token-by-token.

In [28]:
# create input
input = {
    "messages": [
        {
            "role": "human",
            "content": "What's the weather in SF?",
        }
    ]
}

llm_response = ""

# stream token-by-token
async for chunk in client.runs.stream(
    thread_id=thread["thread_id"],
    assistant_id="agent",
    input=input,
    stream_mode="events",
):
    if (
        chunk.event == "events" and
        chunk.data["event"] == "on_chat_model_stream"
    ):
        llm_response += chunk.data["data"]["chunk"]["content"]
        print(llm_response)

b
be
beg
begi
begin
begine
beginen
beginend
